In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
#从datasets下载相应的数据集,并分别风味训练数据与测试数据
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

100%|████████████████████████████████████████████████████████████████| 26421880/26421880 [00:01<00:00, 24263175.06it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 112224.02it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████| 4422102/4422102 [00:03<00:00, 1145986.07it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [ ]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64
#创建一个dataLoader便于后面的采样，批处理等流程
train_dataLoader = DataLoader(training_data , batch_size=batch_size,shuffle=True)
test_dataLoader = DataLoader(test_data , batch_size=batch_size,shuffle=True)



In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# 确定一个模型，在这里必须继承nn.module
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), #这里的输入必须为图像长*宽的特征
            nn.ReLU(),
            nn.Dropout(0.5),  # 防止过拟合
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5), #add 521->256 will quick
            nn.Linear(512, 10)
        )
        # He初始化（ReLU推荐）
        nn.init.kaiming_uniform_(self.linear_relu_stack[0].weight) #对nn.Linear(28*28, 512)第0层，做激活函数方差防止无法收敛
        nn.init.kaiming_uniform_(self.linear_relu_stack[3].weight) #以此类推
        nn.init.kaiming_uniform_(self.linear_relu_stack[6].weight)
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataLoader, model, loss_fn, optimizer)
    test(test_dataLoader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305573  [   64/60000]
loss: 2.289731  [ 6464/60000]
loss: 2.268160  [12864/60000]
loss: 2.262828  [19264/60000]
loss: 2.237950  [25664/60000]
loss: 2.220253  [32064/60000]
loss: 2.225089  [38464/60000]
loss: 2.188047  [44864/60000]
loss: 2.178508  [51264/60000]
loss: 2.153641  [57664/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.142766 

Epoch 2
-------------------------------
loss: 2.149290  [   64/60000]
loss: 2.135898  [ 6464/60000]
loss: 2.077974  [12864/60000]
loss: 2.096091  [19264/60000]
loss: 2.038344  [25664/60000]
loss: 1.985270  [32064/60000]
loss: 2.008924  [38464/60000]
loss: 1.924997  [44864/60000]
loss: 1.920720  [51264/60000]
loss: 1.856866  [57664/60000]
Test Error: 
 Accuracy: 52.2%, Avg loss: 1.855314 

Epoch 3
-------------------------------
loss: 1.884421  [   64/60000]
loss: 1.853034  [ 6464/60000]
loss: 1.737031  [12864/60000]
loss: 1.779356  [19264/60000]
loss: 1.670267  [25664/60000]
loss: 1.630576  [32064/600

In [25]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [27]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
